In [53]:
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from category_encoders import TargetEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression,Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer

In [20]:
from sklearn.compose import make_column_transformer

In [2]:
df = pd.read_csv('winemag-data-130k-v2.csv')

In [4]:
df = df.loc[df.country=='US']

In [6]:
df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
10,10,US,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,87,19.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,Kirkland Signature
12,12,US,"Slightly reduced, this wine offers a chalky, t...",NaN,87,34.0,California,Alexander Valley,Sonoma,Virginie Boone,@vboone,Louis M. Martini 2012 Cabernet Sauvignon (Alex...,Cabernet Sauvignon,Louis M. Martini


In [7]:
print(df.columns)

Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
       'price', 'province', 'region_1', 'region_2', 'taster_name',
       'taster_twitter_handle', 'title', 'variety', 'winery'],
      dtype='object')


In [8]:
df=df.drop(['Unnamed: 0', 'country', 'taster_twitter_handle'], axis=1)

In [9]:
Y = df.points
df_numerical = df.drop(['description'],axis=1)
df_numerical = df_numerical.drop(['points'], axis=1)

In [10]:
df_numerical.nunique()

designation    14183
price            160
province          27
region_1         264
region_2          17
taster_name       15
title          50229
variety          257
winery          5375
dtype: int64

In [11]:
df_numerical.dtypes

designation     object
price          float64
province        object
region_1        object
region_2        object
taster_name     object
title           object
variety         object
winery          object
dtype: object

In [ ]:
#1.1

In [33]:
numerical_features=['price']
onehot_features=['province','region_2','taster_name']
target_features=['designation','region_1','variety','winery']

In [34]:
cont_preprocessor = make_pipeline(SimpleImputer(strategy='median'), StandardScaler())
target_preprocessor = make_pipeline(TargetEncoder(), SimpleImputer(strategy='constant'))
onehot_preprocessor = make_pipeline(SimpleImputer(strategy='constant'), OneHotEncoder(handle_unknown='ignore'))

In [35]:
preprocessor = make_column_transformer((cont_preprocessor, numerical_features),
                                    (target_preprocessor, target_features),
                                    (onehot_preprocessor, onehot_features))

In [38]:
X_train, X_test, y_train, y_test = train_test_split(df_numerical, Y, random_state=1)

In [40]:
ridge_pipe=make_pipeline(preprocessor, Ridge())

In [46]:
import numpy as np
ridge_param_grid = {"ridge__alpha": np.logspace(-3,3,13)}
model=GridSearchCV(ridge_pipe, param_grid = ridge_param_grid, cv = 10)

In [47]:
model.fit(X_train, y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('columntransformer',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('pipeline-1',
                                                                         Pipeline(memory=None,
                                                                                  steps=[('simpleimputer',
                                                                                          SimpleImputer(add_indicator=False,
                                                                                                        copy=True,
                 

In [48]:
print(model.best_params_)
print(model.best_score_)

{'ridge__alpha': 0.001}
0.46308748324427107


In [49]:
print(model.score(X_test, y_test))

0.4603764712220624


In [50]:
# 1.2 Simple Bag of Words Model

In [51]:
textual_feature = df.description

In [54]:
vect = CountVectorizer(min_df=10, stop_words = 'english')

In [56]:
bow = vect.fit_transform(textual_feature)

In [57]:
X_train, X_test, y_train, y_test = train_test_split(bow, Y, random_state=1)

In [58]:
clf = Ridge()
ridge_param_grid = {'alpha': np.logspace(-3,3,13)}
ridge_grid = GridSearchCV(clf, ridge_param_grid, cv=10)
ridge_grid.fit(X_train,y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': array([1.00000000e-03, 3.16227766e-03, 1.00000000e-02, 3.16227766e-02,
       1.00000000e-01, 3.16227766e-01, 1.00000000e+00, 3.16227766e+00,
       1.00000000e+01, 3.16227766e+01, 1.00000000e+02, 3.16227766e+02,
       1.00000000e+03])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [59]:
print(ridge_grid.best_params_)
print(ridge_grid.best_score_)

{'alpha': 10.0}
0.6962269575447063


In [60]:
print(ridge_grid.score(X_test, y_test))

0.6979591487451728


In [61]:
# 1.3 Using Bag of Words with TF-IDF

In [62]:
from sklearn.feature_extraction.text import TfidfTransformer 

In [64]:
text_pipe = make_pipeline(CountVectorizer(min_df=10, stop_words = 'english'),
                             TfidfTransformer())

In [65]:
textual_tfidf = text_pipe.fit_transform(textual_feature)

In [66]:
X_train, X_test, y_train, y_test = train_test_split(textual_tfidf, Y, random_state=1)

In [67]:
clf = Ridge()
ridge_param_grid = {'alpha': np.logspace(-3,3,13)}
ridge_grid = GridSearchCV(clf, ridge_param_grid, cv=10)
ridge_grid.fit(X_train,y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': array([1.00000000e-03, 3.16227766e-03, 1.00000000e-02, 3.16227766e-02,
       1.00000000e-01, 3.16227766e-01, 1.00000000e+00, 3.16227766e+00,
       1.00000000e+01, 3.16227766e+01, 1.00000000e+02, 3.16227766e+02,
       1.00000000e+03])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [68]:
print(ridge_grid.best_params_)
print(ridge_grid.best_score_)

{'alpha': 0.31622776601683794}
0.6995629431059659


In [69]:
print(ridge_grid.score(X_test, y_test))

0.7011738791840928


In [70]:
# 1.3 Using Bag of Words with N-grams

In [89]:
X_train, X_test, y_train, y_test = train_test_split(textual_feature, Y, random_state=1)

In [90]:
from sklearn.preprocessing import Normalizer
param_grid = {"ridge__alpha": np.logspace(-3,3,13) ,
              "countvectorizer__ngram_range": [(1, 1), (1, 2), (1, 3)]
              #"countvectorizer__min_df": [1, 2, 3],
              #"normalizer": [None, Normalizer()]
             }
grid = GridSearchCV(make_pipeline(CountVectorizer(stop_words = 'english', min_df = 3) , Ridge()),
                    param_grid=param_grid, cv=10)

In [91]:
grid.fit(X_train,y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('countvectorizer',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=3,
                                                        ngram_range=(1, 1),
                                                

In [92]:
print(grid.best_params_)
print(grid.best_score_)

{'countvectorizer__ngram_range': (1, 3), 'ridge__alpha': 31.622776601683793}
0.7130953850689815


In [94]:
print(grid.score(X_test, y_test))

0.7160507062509373


In [95]:
# 1.4 - Combining textual and non-textual features

In [96]:
df_numerical['description'] = list(textual_feature)

In [97]:
df_numerical.head

<bound method NDFrame.head of                                designation  price    province  \
2                                      NaN   14.0      Oregon   
3                     Reserve Late Harvest   13.0    Michigan   
4       Vintner's Reserve Wild Child Block   65.0      Oregon   
10                          Mountain Cuvée   19.0  California   
12                                     NaN   34.0  California   
...                                    ...    ...         ...   
129945    Jurassic Park Vineyard Old Vines   20.0  California   
129949                    Barrel Fermented   35.0  California   
129950                       Blocks 7 & 22   35.0  California   
129952                                 NaN   22.0  California   
129967                                 NaN   75.0      Oregon   

                   region_1           region_2         taster_name  \
2         Willamette Valley  Willamette Valley        Paul Gregutt   
3       Lake Michigan Shore                NaN  A

In [119]:
numerical_features=['price']
onehot_features=['province','region_2','taster_name']
target_features=['designation','region_1','variety','winery']
text_features = 'description'

In [120]:
# n-gram range is taken as (1,3) based on the best parameters obtained in the previous model 
cont_preprocessor = make_pipeline(SimpleImputer(strategy='median'), StandardScaler())
target_preprocessor = make_pipeline(TargetEncoder(), SimpleImputer(strategy='constant'))
onehot_preprocessor = make_pipeline(SimpleImputer(strategy='constant'), OneHotEncoder(handle_unknown='ignore'))
text_preprocessor = make_pipeline(CountVectorizer(stop_words = 'english', ngram_range = (1,3)))

In [121]:
preprocessor = make_column_transformer((cont_preprocessor, numerical_features),
                                    (target_preprocessor, target_features),
                                    (onehot_preprocessor, onehot_features),
                                      (text_preprocessor, text_features))

In [122]:
X_train, X_test, y_train, y_test = train_test_split(df_numerical, Y, random_state=1)

In [123]:
ridge_pipe=make_pipeline(preprocessor, Ridge())

In [124]:
import numpy as np
ridge_param_grid = {"ridge__alpha": np.logspace(-3,3,13)}
model=GridSearchCV(ridge_pipe, param_grid = ridge_param_grid, cv = 10)

In [125]:
model.fit(X_train, y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('columntransformer',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('pipeline-1',
                                                                         Pipeline(memory=None,
                                                                                  steps=[('simpleimputer',
                                                                                          SimpleImputer(add_indicator=False,
                                                                                                        copy=True,
                 

In [126]:
print(model.best_params_)
print(model.best_score_)

{'ridge__alpha': 3.1622776601683795}
0.7828374196599628


In [127]:
print(model.score(X_test, y_test))

0.7881934505501371


It can be seen that by taking both textual and non-textual features, the score (0.78) has improved over the best possible score obtained using just a bag of words model for the textual features (0.71)